In [ ]:
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.offline as offline
import numpy as np
import time
import copy
import datetime

In [ ]:
#有 n 個工件以及 m 台機台，每個工件在機台的加工順序都一樣
#目標為最小化總加權延遲

''' ================= initialization setting ======================'''

num_job=20 # number of jobs

p=[10,10,13,4,9,4,8,15,7,1,9,3,15,9,11,6,5,14,18,3] #完工時間
d=[50,38,49,12,20,105,73,45,6,64,15,6,92,43,78,21,15,50,150,99] #到期日
w=[10,5,1,5,10,1,5,10,5,1,5,10,10,5,1,10,5,5,1,5] #權重

population_size=int(input('Please input the size of population: ') or 30) # default value is 30
crossover_rate=float(input('Please input the size of Crossover Rate: ') or 0.8) # default value is 0.8
mutation_rate=float(input('Please input the size of Mutation Rate: ') or 0.1) # default value is 0.1
mutation_selection_rate=float(input('Please input the mutation selection rate: ') or 0.5)
num_mutation_jobs=round(num_job*mutation_selection_rate)
num_iteration=int(input('Please input number of iteration: ') or 2000) # default value is 2000


start_time = time.time() #計算所花費時間  

'''==================== main code ==============================='''
'''----- generate initial population -----'''
Tbest=999999999999999 #設定停止條件
best_list,best_obj=[],[] #存放最佳解
population_list=[] #存放產生的所有解
for i in range(population_size):
    random_num=list(np.random.permutation(num_job)) # 生成0到num_job-1的隨機排列
    population_list.append(random_num) # 增加至population_list
        
for n in range(num_iteration): # 開始進行演化，自訂要跑幾代
    Tbest_now=99999999999  #設定預計預計停止的分數    

    '''-------- crossover --------'''
    parent_list=copy.deepcopy(population_list) #父母代基因
    offspring_list=copy.deepcopy(population_list) #子代基因
    S=list(np.random.permutation(population_size)) # 生成隨機序列，選擇要交叉的父染色體
    
    for m in range(int(population_size/2)): #在這迴圈中會挑選兩個父母代進行交配，進行population size /2 次
        crossover_prob=np.random.rand() #隨機生成交配率
        if crossover_rate>=crossover_prob: #根據交配率來決定是否要進行交配
            parent_1= population_list[S[2*m]][:] #挑選第一個要進行交配的父母代
            parent_2= population_list[S[2*m+1]][:] #挑選第二個要進行交配的父母代 
            child_1=['na' for i in range(num_job)] #為了後續交配準備
            child_2=['na' for i in range(num_job)] #為了後續交配準備
            fix_num=round(num_job/2) #將基因數一半的位置設為固定不變
            g_fix=list(np.random.choice(num_job, fix_num, replace=False)) #除了固定不變基因，剩餘基因隨機挑選交配
            
            for g in range(fix_num):
                child_1[g_fix[g]]=parent_2[g_fix[g]] #將Parent_2不變的位置複製，產生child_1 
                child_2[g_fix[g]]=parent_1[g_fix[g]] #將Parent_1不變的位置複製，產生child_2 
            c1=[parent_1[i] for i in range(num_job) if parent_1[i] not in child_1] #c1為，parent_1與child_1不重複的工作
            c2=[parent_2[i] for i in range(num_job) if parent_2[i] not in child_2] #c2為，parent_2與child_2不重複的工作
            
            for i in range(num_job-fix_num):
                child_1[child_1.index('na')]=c1[i] #將parent_1與child_1不重複的工作c1依序填入空缺位置
                child_2[child_2.index('na')]=c2[i] #將parent_2與child_2不重複的工作c2依序填入空缺位置
            offspring_list[S[2*m]]=child_1[:]
            offspring_list[S[2*m+1]]=child_2[:]
            #交配結束，產生兩個新的子代
        
    '''--------mutatuon--------'''   
    #透過基因位移的方式進行突變
    for m in range(len(offspring_list)):
        mutation_prob=np.random.rand() #決定染色體中有多少百分比的基因要進行突變
        if mutation_rate >= mutation_prob: #當設定的突變率>=隨機生成的突變率將進行突變
            m_chg=list(np.random.choice(num_job, num_mutation_jobs, replace=False)) #隨機選擇突變的位置
            t_value_last=offspring_list[m][m_chg[0]] #保存第一個突變位置上的值
            for i in range(num_mutation_jobs-1): #此方式為位移，因此進行 num_mutation_jobs-1次
                offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] #突變位置的值向右移一位
            
            offspring_list[m][m_chg[num_mutation_jobs-1]]=t_value_last #將第一個突變位置的值覆蓋到最後一個突變位置
    
    '''--------fitness value(calculate tardiness)-------------'''
    ###不太了解####
    
    total_chromosome=copy.deepcopy(parent_list)+copy.deepcopy(offspring_list) #全部的基因=父母和子代結合
    chrom_fitness,chrom_fit=[],[] #存放適應值, 存放原始時間
    total_fitness=0 #對整個群體進行評分
    for i in range(population_size*2):
        ptime=0
        tardiness=0
        for j in range(num_job):
            ptime=ptime+p[total_chromosome[i][j]] #累計時間
            tardiness=tardiness+w[total_chromosome[i][j]]*max(ptime-d[total_chromosome[i][j]],0) #累計延遲時間
        chrom_fitness.append(1/tardiness)
        chrom_fit.append(tardiness)
        total_fitness=total_fitness+chrom_fitness[i]
    
    '''----------selection----------'''
    pk,qk=[],[] #存放選擇幾率pk及累積機率qk值
    
    for i in range(population_size*2): #計算所有要被選擇染色體的適應值總和
        pk.append(chrom_fitness[i]/total_fitness) #對每個染色體計算其選擇機率pk 計算分數佔比
    for i in range(population_size*2): #計算所有要被選擇染色體的適應值總和
        cumulative=0 #對每個染色體計算其累積機率qk
        for j in range(0,i+1):
            cumulative=cumulative+pk[j]
        qk.append(cumulative)
    
    selection_rand=[np.random.rand() for i in range(population_size)] #從[0,1]區間隨機產生一個數字
    
    for i in range(population_size):
        #若隨機產生的數字<=qK1，則選擇第一條染色體
        if selection_rand[i]<=qk[0]:
            population_list[i]=copy.deepcopy(total_chromosome[0])
        #否則當qk-1<隨機產生的數字<qk，則選擇第i個染色體
        else:
            for j in range(0,population_size*2-1):
                if selection_rand[i]>qk[j] and selection_rand[i]<=qk[j+1]:
                    population_list[i]=copy.deepcopy(total_chromosome[j+1])
                    break #直到選擇的染色體數量達到所設定族群大小，則停止
                
    '''----------comparison----------'''
    #比較每個染色體的總加權延遲 chrom_fit 
    for i in range(population_size*2):
        #如果chrom_fit內的時間小於我們預設的最佳時間，就以Tbest_now儲存
        if chrom_fit[i]<Tbest_now:
            Tbest_now=chrom_fit[i]
            sequence_now=copy.deepcopy(total_chromosome[i]) #存目前最佳的排程
    
    if Tbest_now<=Tbest: #若此解比最佳解還好
        Tbest=Tbest_now #就會替代Tbest
        sequence_best=copy.deepcopy(sequence_now) #存目前最佳的排程
    
    ###不太了解####
    #計算每個工件的延遲時間
    job_sequence_ptime=0 #工作所需時間
    num_tardy=0 #工作延遲
    for k in range(num_job): 
        job_sequence_ptime=job_sequence_ptime+p[sequence_best[k]] 
        if job_sequence_ptime>d[sequence_best[k]]: 
            num_tardy=num_tardy+1 

'''----------result----------'''
print("optimal sequence",sequence_best) #最好排程結果
print("optimal value:%f"%Tbest) #該排程總加權延遲時間
print("average tardiness:%f"%(Tbest/num_job)) #每個工件平均加權延遲時間
print("number of tardy:%d"%num_tardy) #有多少工件延遲
print('the elapsed time:%s'% (time.time() - start_time)) #顯示總運算時間

In [ ]:

'''--------plot gantt chart-------'''
#甘特圖

j_keys=[j for j in range(num_job)]
j_count={key:0 for key in j_keys}
m_count=0
j_record={}
for i in sequence_best:
   gen_t=int(p[i])
   j_count[i]=j_count[i]+gen_t
   m_count=m_count+gen_t
   
   if m_count<j_count[i]:
       m_count=j_count[i]
   elif m_count>j_count[i]:
       j_count[i]=m_count
   start_time=str(datetime.timedelta(seconds=j_count[i]-p[i])) # convert seconds to hours, minutes and seconds

   end_time=str(datetime.timedelta(seconds=j_count[i]))
   j_record[i]=[start_time,end_time]
       

df=[]
for j in j_keys:
   df.append(dict(Task='Machine', Start='2018-07-14 %s'%(str(j_record[j][0])), Finish='2018-07-14 %s'%(str(j_record[j][1])),Resource='Job %s'%(j+1)))

# colors={}
# for i in j_keys:
#     colors['Job %s'%(i+1)]='rgb(%s,%s,%s)'%(255/(i+1)+0*i,5+12*i,50+10*i)

fig = ff.create_gantt(df, colors=['#008B00','#FF8C00','#E3CF57','#0000CD','#7AC5CD','#ED9121','#76EE00','#6495ED','#008B8B','#A9A9A9','#A2CD5A','#9A32CD','#8FBC8F','#EEC900','#EEE685','#CDC1C5','#9AC0CD','#EEA2AD','#00FA9A','#CDB38B'], index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True)
py.iplot(fig, filename='GA_flow_shop_scheduling_tardyjob', world_readable=True)